In [1]:
from biobert_embedding.embedding import BiobertEmbedding
from scipy.spatial import distance
import pandas as pd
import os
from scipy.spatial import distance

2023-04-29 02:31:17.900507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 02:31:17.950089: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 02:31:17.951495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 02:31:18.642444: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
type_of_ner = "new"

ner_df = pd.read_pickle("data/ner_df.p")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")

In [3]:
null_index_list = []
for i in ner_df.itertuples():
    if len(i.ner) == 0:
        null_index_list.append(i.Index)
ner_df.drop(null_index_list, inplace=True)

In [4]:
print(len(ner_df))

132887


In [5]:
def handle_truncate_list(key, sentence_vector, my_dict, max_items_in_list = 100):
    if key in my_dict: 
        
        # we must do some logic to remove the most similar item
        if len(my_dict[key]) == max_items_in_list: 
            my_list  = my_dict[key]
            similarity_value = 0.00
            index = -1
            for i, vector_to_compare in enumerate(my_list):
                cosine_sim = 1 - distance.cosine(sentence_vector, vector_to_compare)
                if cosine_sim > similarity_value:
                    index = i
                    similarity_value = cosine_sim
            if index > -1:
                my_dict[key].pop(index)
    else:
        my_dict[key] = list()
    
    my_dict[key].append(sentence_vector.detach().numpy())

In [6]:
count = 0
biobert = BiobertEmbedding("/home/ubuntu/biobertmodel")

new_biobert_dict = {}
with open("/home/ubuntu/bad_vectors.txt", "w") as file1:
    # Writing data to a file
    sentence_vector = ""
    num_exceptions = 0
    for i in ner_df.itertuples():
        # print(i)
        
        # we should only look for ids that are relevant
        if i.SUBJECT_ID not in ner_fasttext:
            continue
            
        if count % 1000 == 0:
            print(f"processed {count} with exceptions: {num_exceptions}")
        if i.preprocessed_text is not None and len(i.preprocessed_text[0]) > 0:
            try:
                # print(i.preprocessed_text[0])
                sentence_vector = biobert.sentence_vector(i.preprocessed_text[0])
                handle_truncate_list(i.SUBJECT_ID, sentence_vector, new_biobert_dict)
                #if i.SUBJECT_ID not in new_biobert_dict:
                 #   new_biobert_dict[i.SUBJECT_ID] = list(sentence_vector)            
                #else:
                 #   new_biobert_dict[i.SUBJECT_ID].append(sentence_vector) 
            except Exception as e:
                print(str(e))
                num_exceptions += 1
                file1.write(f"error on subject: {i.SUBJECT_ID}    {i.preprocessed_text[0]}" + "\n")
                unique_keywords = dict()
                to_sentence = list()
                for list_of_tuples in i.ner:
                    for a_tuple in list_of_tuples:
                        if a_tuple[0] not in unique_keywords:
                            unique_keywords[a_tuple[0]] = a_tuple[1]
                            to_sentence.append(a_tuple[0])
                            to_sentence.append(a_tuple[1])
                
                to_sentence = [word.strip() for word in to_sentence]
                sentence_vector = biobert.sentence_vector(' '.join(to_sentence))
                
                handle_truncate_list(i.SUBJECT_ID, sentence_vector, new_biobert_dict)
                
                #if i.SUBJECT_ID not in new_biobert_dict:
                 #   new_biobert_dict[i.SUBJECT_ID] = list(' '.join(to_sentence))            
                #else:
                #    new_biobert_dict[i.SUBJECT_ID].append(' '.join(to_sentence)) 
                    
        count += 1
    



processed 0 with exceptions: 0
index out of range in self
processed 1000 with exceptions: 1
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
processed 2000 with exceptions: 6
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
processed 3000 with exceptions: 16
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
index out of range in self
processed 4000 with exceptions: 25
index out of range in self
index out of range in self
index out of range in self
index out of range in self
processed 5000 with exceptions: 29
index out of 

In [7]:
test = [[('heparin', 'DRUG'), ('drip', 'ROUTE')], [('niacin', 'DRUG'), ('naprosyn', 'DRUG'), ('naproxen', 'DRUG')], [('2l', 'DOSAGE')], [('zofran', 'DRUG'), ('ativan', 'DRUG')]]
# biobert = BiobertEmbedding("/home/ubuntu/biobertmodel")
unique_keywords = dict()
to_sentence = list()
for list_of_tuples in test:
    for a_tuple in list_of_tuples:
        if a_tuple[0] not in unique_keywords:
            unique_keywords[a_tuple[0]] = a_tuple[1]
            to_sentence.append(a_tuple[0])
            to_sentence.append(a_tuple[1])
                
            to_sentence = [word.strip() for word in to_sentence]
print(' '.join(to_sentence))
# print(biobert.sentence_vector(' '.join(to_sentence)).shape)

heparin DRUG drip ROUTE niacin DRUG naprosyn DRUG naproxen DRUG 2l DOSAGE zofran DRUG ativan DRUG


In [8]:
pd.to_pickle(new_biobert_dict, "data/biobert_test_data.pkl")

In [9]:
len(list(new_biobert_dict.keys()))

10000